In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1252505,2021-05-07,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1252506,2021-05-08,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1252507,2021-05-09,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1252508,2021-05-10,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
33399,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33401,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33403,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33405,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
33407,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
770232,2021-05-07,Arkansas,Arkansas,5001,2098,36.00,5000,Arkansas,AR,Arkansas,State,3017804
770234,2021-05-08,Arkansas,Arkansas,5001,2101,36.00,5000,Arkansas,AR,Arkansas,State,3017804
770236,2021-05-09,Arkansas,Arkansas,5001,2101,36.00,5000,Arkansas,AR,Arkansas,State,3017804
770238,2021-05-10,Arkansas,Arkansas,5001,2101,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1252505,2021-05-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1252506,2021-05-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1252507,2021-05-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1252508,2021-05-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-05-11') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
476,2021-05-11,Snohomish,Washington,37276,573.00,53061,WA,County,822083,4534.34,69.70
950,2021-05-11,Cook,Illinois,543658,10635.00,17031,IL,County,5150233,10555.99,206.50
1423,2021-05-11,Orange,California,270883,5017.00,6059,CA,County,3175692,8529.89,157.98
1895,2021-05-11,Maricopa,Arizona,542486,9932.00,4013,AZ,County,4485414,12094.45,221.43
2367,2021-05-11,Los Angeles,California,1236634,24030.00,6037,CA,County,10039107,12318.17,239.36
...,...,...,...,...,...,...,...,...,...,...,...
1251789,2021-05-11,Wheeler,Oregon,30,1.00,41069,OR,County,1332,2252.25,75.08
1252000,2021-05-11,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1252180,2021-05-11,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1252356,2021-05-11,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
476,2021-05-11,Snohomish,Washington,37276,573.00,53061,WA,County,822083,4534.34,69.70,69.70,4534.34
950,2021-05-11,Cook,Illinois,543658,10635.00,17031,IL,County,5150233,10555.99,206.50,206.50,10555.99
1423,2021-05-11,Orange,California,270883,5017.00,6059,CA,County,3175692,8529.89,157.98,157.98,8529.89
1895,2021-05-11,Maricopa,Arizona,542486,9932.00,4013,AZ,County,4485414,12094.45,221.43,221.43,12094.45
2367,2021-05-11,Los Angeles,California,1236634,24030.00,6037,CA,County,10039107,12318.17,239.36,239.36,12318.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1251789,2021-05-11,Wheeler,Oregon,30,1.00,41069,OR,County,1332,2252.25,75.08,75.08,2252.25
1252000,2021-05-11,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1252180,2021-05-11,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1252356,2021-05-11,Loving,Texas,2,0.00,48301,TX,County,169,1183.43,0.00,0.00,1183.43


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
805411,2021-05-11,Hale,Alabama,2230,77.00,1065,AL,County,14651,15220.80,525.56,525.56,15220.80,1
876731,2021-05-11,Clarke,Alabama,3511,61.00,1025,AL,County,23622,14863.26,258.23,258.23,14863.26,2
630081,2021-05-11,Lowndes,Alabama,1388,53.00,1085,AL,County,9726,14271.03,544.93,544.93,14271.03,3
468726,2021-05-11,Franklin,Alabama,4260,82.00,1059,AL,County,31362,13583.32,261.46,261.46,13583.32,4
558638,2021-05-11,Etowah,Alabama,13843,353.00,1055,AL,County,102268,13536.00,345.17,345.17,13536.00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1186863,2021-05-11,Platte,Wyoming,625,11.00,56031,WY,County,8393,7446.68,131.06,131.06,7446.68,19
768603,2021-05-11,Converse,Wyoming,1002,17.00,56009,WY,County,13822,7249.31,122.99,122.99,7249.31,20
955708,2021-05-11,Lincoln,Wyoming,1409,12.00,56023,WY,County,19830,7105.40,60.51,60.51,7105.40,21
1063794,2021-05-11,Niobrara,Wyoming,161,2.00,56027,WY,County,2356,6833.62,84.89,84.89,6833.62,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
630081,2021-05-11,Lowndes,Alabama,1388,53.00,1085,AL,County,9726,14271.03,544.93,544.93,14271.03,3,1
805411,2021-05-11,Hale,Alabama,2230,77.00,1065,AL,County,14651,15220.80,525.56,525.56,15220.80,1,2
267869,2021-05-11,Walker,Alabama,7241,277.00,1127,AL,County,63521,11399.38,436.08,436.08,11399.38,28,3
556986,2021-05-11,Clay,Alabama,1567,56.00,1027,AL,County,13235,11839.82,423.12,423.12,11839.82,19,4
679518,2021-05-11,Greene,Alabama,925,34.00,1063,AL,County,8111,11404.27,419.18,419.18,11404.27,27,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677873,2021-05-11,Hot Springs,Wyoming,369,3.00,56017,WY,County,4413,8361.66,67.98,67.98,8361.66,17,19
931872,2021-05-11,Uinta,Wyoming,2232,13.00,56041,WY,County,20226,11035.30,64.27,64.27,11035.30,5,20
955708,2021-05-11,Lincoln,Wyoming,1409,12.00,56023,WY,County,19830,7105.40,60.51,60.51,7105.40,21,21
266612,2021-05-11,Teton,Wyoming,3758,9.00,56039,WY,County,23464,16016.02,38.36,38.36,16016.02,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.70,4534.34,19,24
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.70,4534.34,19,24
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.70,4534.34,19,24
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.70,4534.34,19,24
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,69.70,4534.34,19,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1249572,2021-05-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1249573,2021-05-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1249574,2021-05-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1249575,2021-05-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
802071,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15220.80,2,1,1.00
802072,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15220.80,2,1,0.00
802073,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15220.80,2,1,0.00
802074,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,525.56,15220.80,2,1,0.00
802075,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,525.56,15220.80,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971712,2021-05-07,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5696.20,6,23,0.00
971713,2021-05-08,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5696.20,6,23,0.00
971714,2021-05-09,Crook,Wyoming,429,12.00,56011,WY,County,7584,5656.65,158.23,158.23,5696.20,6,23,0.00
971715,2021-05-10,Crook,Wyoming,431,12.00,56011,WY,County,7584,5683.02,158.23,158.23,5696.20,6,23,2.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
627557,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14271.03,1,3,1.00,0.00
627558,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14271.03,1,3,0.00,0.00
627559,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14271.03,1,3,0.00,0.00
627560,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14271.03,1,3,0.00,0.00
627561,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14271.03,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625492,2021-05-07,Albany,Wyoming,4329,12.00,56001,WY,County,38880,11134.26,30.86,30.86,11247.43,23,4,27.00,0.00
625493,2021-05-08,Albany,Wyoming,4329,12.00,56001,WY,County,38880,11134.26,30.86,30.86,11247.43,23,4,0.00,0.00
625494,2021-05-09,Albany,Wyoming,4329,12.00,56001,WY,County,38880,11134.26,30.86,30.86,11247.43,23,4,0.00,0.00
625495,2021-05-10,Albany,Wyoming,4365,12.00,56001,WY,County,38880,11226.85,30.86,30.86,11247.43,23,4,36.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-05-11') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
319793,2021-05-11,Imperial,California,28397,722.00,6025,CA,County,181215,15670.34,398.42,398.42,15670.34,1,2,9.00,1.00
2367,2021-05-11,Los Angeles,California,1236634,24030.00,6037,CA,County,10039107,12318.17,239.36,239.36,12318.17,2,5,173.00,4.00
159193,2021-05-11,San Bernardino,California,296700,4707.00,6071,CA,County,2180085,13609.56,215.91,215.91,13609.56,3,4,45.00,0.00
560242,2021-05-11,Inyo,California,1428,38.00,6027,CA,County,18039,7916.18,210.65,210.65,7916.18,4,25,2.00,0.00
71165,2021-05-11,Stanislaus,California,62058,1058.00,6099,CA,County,550660,11269.75,192.13,192.13,11269.75,5,9,50.00,2.00
32967,2021-05-11,Riverside,California,299376,4590.00,6065,CA,County,2470546,12117.81,185.79,185.79,12117.81,6,7,203.00,2.00
56615,2021-05-11,San Joaquin,California,73180,1391.00,6077,CA,County,762148,9601.81,182.51,182.51,9601.81,7,18,6.00,0.00
88242,2021-05-11,Tulare,California,49613,839.00,6107,CA,County,466195,10642.11,179.97,179.97,10642.11,8,10,34.00,0.00
32105,2021-05-11,Fresno,California,101755,1682.00,6019,CA,County,999101,10184.66,168.35,168.35,10184.66,9,14,26.00,0.00
507681,2021-05-11,Merced,California,31859,460.00,6047,CA,County,277680,11473.28,165.66,165.66,11473.28,10,8,38.00,1.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1186414,2021-05-11,Lassen,California,5701,24.00,6035,CA,County,30573,18647.17,78.50,78.50,18647.17,34,1,3.00,0.00
319793,2021-05-11,Imperial,California,28397,722.00,6025,CA,County,181215,15670.34,398.42,398.42,15670.34,1,2,9.00,1.00
679460,2021-05-11,Kings,California,22981,246.00,6031,CA,County,152940,15026.15,160.85,160.85,15026.15,11,3,4.00,0.00
159193,2021-05-11,San Bernardino,California,296700,4707.00,6071,CA,County,2180085,13609.56,215.91,215.91,13609.56,3,4,45.00,0.00
2367,2021-05-11,Los Angeles,California,1236634,24030.00,6037,CA,County,10039107,12318.17,239.36,239.36,12318.17,2,5,173.00,4.00
200169,2021-05-11,Kern,California,109101,1373.00,6029,CA,County,900202,12119.61,152.52,152.52,12119.61,14,6,49.00,0.00
32967,2021-05-11,Riverside,California,299376,4590.00,6065,CA,County,2470546,12117.81,185.79,185.79,12117.81,6,7,203.00,2.00
507681,2021-05-11,Merced,California,31859,460.00,6047,CA,County,277680,11473.28,165.66,165.66,11473.28,10,8,38.00,1.00
71165,2021-05-11,Stanislaus,California,62058,1058.00,6099,CA,County,550660,11269.75,192.13,192.13,11269.75,5,9,50.00,2.00
88242,2021-05-11,Tulare,California,49613,839.00,6107,CA,County,466195,10642.11,179.97,179.97,10642.11,8,10,34.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
319793,2021-05-11,Imperial,California,28397,722.00,6025,CA,County,181215,15670.34,398.42,398.42,15670.34,1,2,9.00,1.00
2367,2021-05-11,Los Angeles,California,1236634,24030.00,6037,CA,County,10039107,12318.17,239.36,239.36,12318.17,2,5,173.00,4.00
159193,2021-05-11,San Bernardino,California,296700,4707.00,6071,CA,County,2180085,13609.56,215.91,215.91,13609.56,3,4,45.00,0.00
560242,2021-05-11,Inyo,California,1428,38.00,6027,CA,County,18039,7916.18,210.65,210.65,7916.18,4,25,2.00,0.00
71165,2021-05-11,Stanislaus,California,62058,1058.00,6099,CA,County,550660,11269.75,192.13,192.13,11269.75,5,9,50.00,2.00
32967,2021-05-11,Riverside,California,299376,4590.00,6065,CA,County,2470546,12117.81,185.79,185.79,12117.81,6,7,203.00,2.00
56615,2021-05-11,San Joaquin,California,73180,1391.00,6077,CA,County,762148,9601.81,182.51,182.51,9601.81,7,18,6.00,0.00
88242,2021-05-11,Tulare,California,49613,839.00,6107,CA,County,466195,10642.11,179.97,179.97,10642.11,8,10,34.00,0.00
32105,2021-05-11,Fresno,California,101755,1682.00,6019,CA,County,999101,10184.66,168.35,168.35,10184.66,9,14,26.00,0.00
507681,2021-05-11,Merced,California,31859,460.00,6047,CA,County,277680,11473.28,165.66,165.66,11473.28,10,8,38.00,1.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,398.42,15670.34,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,398.42,15670.34,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,398.42,15670.34,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,398.42,15670.34,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,398.42,15670.34,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4192,05/07/21,Lassen,5697,24.00,18634.09,78.50,78.50,18647.17,34,1,1.00,0.00
4193,05/08/21,Lassen,5698,24.00,18637.36,78.50,78.50,18647.17,34,1,1.00,0.00
4194,05/09/21,Lassen,5698,24.00,18637.36,78.50,78.50,18647.17,34,1,0.00,0.00
4195,05/10/21,Lassen,5698,24.00,18637.36,78.50,78.50,18647.17,34,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,398.42,15670.34,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,398.42,15670.34,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,398.42,15670.34,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,398.42,15670.34,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,398.42,15670.34,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4278,05/07/21,Merced,31743,458.00,11431.50,164.94,165.66,11473.28,10,8,32.00,2.00
4279,05/08/21,Merced,31743,458.00,11431.50,164.94,165.66,11473.28,10,8,0.00,0.00
4280,05/09/21,Merced,31743,458.00,11431.50,164.94,165.66,11473.28,10,8,0.00,0.00
4281,05/10/21,Merced,31821,459.00,11459.59,165.30,165.66,11473.28,10,8,78.00,1.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)